# ПОКА В ПРОЦЕССЕ РАЗРАБОТКЕ

In [1]:
!pip install telethon 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/702.2 kB ? eta -:--:--
   ---------------------------------------- 702.2/702.2 kB 7.0 MB/s eta 0:00:00
Using cached rsa-4.9-py3-none-any.whl (34 kB)
Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26358 sha256=c27b43143e75f20e00d07726c6809b428a556ba01cb543777d57d7425743bb1d
  Stored in directory: c:\users\vallo\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\4e\52\3

In [ ]:
import asyncio
from telethon import TelegramClient
from telethon.tl.types import InputPeerChannel
from telethon.tl.functions.channels import GetFullChannelRequest
import logging
import json
from datetime import datetime, timedelta

# Настройки подключения
API_ID = 'ваш_api_id'
API_HASH = 'ваш_api_hash'
PHONE_NUMBER = 'ваш_номер_телефона'

# Список каналов для парсинга
CHANNELS = [
    'channel1',
    'channel2', 
    'channel3'
]

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TelegramParser:
    def __init__(self, api_id, api_hash, phone_number):
        self.client = TelegramClient(phone_number, api_id, api_hash)
        self.parsed_messages = []

    async def connect_client(self):
        await self.client.start()
        logger.info("Клиент подключен")

    async def parse_channel(self, channel_username, limit=100):
        try:
            # Получаем информацию о канале
            channel_entity = await self.client.get_entity(channel_username)
            
            # Получаем сообщения
            messages = await self.client.get_messages(
                channel_entity, 
                limit=limit
            )

            channel_messages = []
            for message in messages:
                # Обработка каждого сообщения
                parsed_message = {
                    'channel': channel_username,
                    'message_id': message.id,
                    'text': message.text,
                    'date': message.date.isoformat() if message.date else None,
                    'views': message.views,
                    'forwards': message.forwards
                }
                channel_messages.append(parsed_message)

            return channel_messages

        except Exception as e:
            logger.error(f"Ошибка при парсинге канала {channel_username}: {e}")
            return []

    async def parse_multiple_channels(self, channels):
        # Асинхронный парсинг нескольких каналов
        tasks = []
        for channel in channels:
            task = asyncio.create_task(self.parse_channel(channel))
            tasks.append(task)
        
        # Ожидаем результаты всех задач
        results = await asyncio.gather(*tasks)
        
        # Flatten результатов
        for channel_messages in results:
            self.parsed_messages.extend(channel_messages)

    async def save_results(self):
        # Сохранение результатов в JSON
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"telegram_news_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.parsed_messages, f, ensure_ascii=False, indent=2)
        
        logger.info(f"Результаты сохранены в {filename}")

    async def run(self):
        try:
            # Подключение клиента
            await self.connect_client()
            
            # Парсинг каналов
            await self.parse_multiple_channels(CHANNELS)
            
            # Сохранение результатов
            await self.save_results()

        except Exception as e:
            logger.error(f"Общая ошибка: {e}")
        finally:
            # Закрытие соединения
            await self.client.disconnect()

async def main():
    # Создание парсера
    parser = TelegramParser(API_ID, API_HASH, PHONE_NUMBER)
    
    # Запуск парсинга
    await parser.run()

    # Задержка между парсингами для избежания бана
    await asyncio.sleep(3600)  # Пауза 1 час

if __name__ == '__main__':
    asyncio.run(main())